# Notebook to illustrate the CANUCS SC#2 pipeline

**CANUCS SC#2:** Probing the low-mass SFR-M$_*$ correlation. This is designed to work with 4 different data-inputs, ranging from the simplest case of photometry to the hardest case of spatially resolved spectrophotometry.

This notebook has been written to be a modular template that can be adapted to the other science cases.

- Created: 7 Feb 2021
- Last Modified: 10 Feb 2021
- Last Modified by: KIyer
- Git repo: `N/A`
- Branch: `N/A`
- Module Dependencies: Photometry, Spectroscopy, Lensing, Morphology

In [ ]:
# Import packages
import numpy as np
import canucs_analysis as canucs

# Print package versions
print('using versions:')
print('numpy: ',np.__version__)
print('canucs_analysis: ',canucs.__version__)

# Import the (spectro)photometry to be fit

In [ ]:
phot_catalog = canucs.photometry.generate_photometry(mosaic, 
                                                     extraction_params, 
                                                     psf_params, 
                                                     cube = False, 
                                                     cube_params = [])

spec_catalog = canucs.spectroscopy.grizli(magic)

In [ ]:
photometry, errors, idpz = canucs.photometry.load_photometry(phot_catalog_filename, cube = False)
spectra, errors, idsz = canucs.spectroscopy.load_spectra(phot_catalog_filename, cube = False)

# idpz and idsz contain RA, DEC and other identifiers to help match the spectrophotometry
sed_catalog = canucs.create_catalog(photometry, errors, idpz, spectra, errors, idsz)

# Initialize the fitter and do any preprocessing necessary

In [ ]:
# model_params - dict[SPS model, SFH(t), Z(t), IMF, Dust, IGM, fitting code or sampling technique, convergence_criterion, parallelize]
# prior_params - dict[redshift_range, SFH_params, Z(t)_params, dust_params, ] 
# dixed_params - dict(IGM = madau, IMF = chabrier, SSP = BC03 or MILES/MIST or BPASS)

fitter = canucs.sedfit.initiate_fitter(model_params, 
                                       fixed_params, 
                                       phot = True, 
                                       spec = True, 
                                       fitter_name, 
                                       cube = False)

priors = canucs.sedfit.initiate_priors(prior_params)

# generates atlas if required by method for computational speedup
canucs.sedfit.preprocessing(N_atlas, N_cores)

#---------- within preprocessing, there are modules to generate SEDs/spectra from SPS codes --------
# prior_sample = priors.sample(random_seed)
# spec = canucs.sedfit.generate_spectra(spectra_params, units)
# sed = sedfit.multiply_by_filters(spec, redshift, filter_list, units)
#---------------------------------------------------------------------------------------------------

# Fit the catalog

In [ ]:
# the heirarchical flag decides whether the fits occur simultaneously or one after the other.
canucs.sedfit.fit_catalog(sed_catalog, fitter, priors, fit_params, heirarchical = False, store = storepath)

#---------- within fit_catalog, there are modules to -------------------
# --------- run fits on individual objects and store posteriors --------
# posteriors = canucs.sedfit.fit_photometry(sed_obs, sed_err, fitter, priors, fit_args)
# posteriors = canucs.sedfit.fit_spectra(sed_obs, sed_err, fitter, priors, fit_args)
# 
#---------------------------------------------------------------------

# Science after fitting

- load outputs
- calculate slope, norm, scatter of SFR-M* correlation
- make completeness corrections